In [39]:
#!pip install pandas
#!pip install xgboost

In [77]:
# Imporrt Libraries
import pandas as pd

In [78]:
# Load Data
df_hospital = pd.read_csv("./Data/Hospital_Dataset_2020_2024.csv")
df_air = pd.read_csv("./Data/Air_Quality - بيانات جودة الهواء في السعودية.csv")

In [79]:
# Display Basic Information
# Display basic info and first few rows
df_air_info = df_air.info()
df_hospital_info = df_hospital.info()

df_air_head = df_air.head()
df_hospital_head = df_hospital.head()

df_air_info, df_air_head, df_hospital_info, df_hospital_head

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120000 entries, 0 to 119999
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   date               120000 non-null  object 
 1   city               120000 non-null  object 
 2   station            120000 non-null  object 
 3   component          120000 non-null  object 
 4   value              120000 non-null  int64  
 5   unit               120000 non-null  object 
 6   indicator          120000 non-null  object 
 7   periodicity        120000 non-null  object 
 8   quarter            120000 non-null  float64
 9   temperature (Â0C)  120000 non-null  float64
 10  humidity (%)       120000 non-null  float64
 11  wind_speed (km/h)  120000 non-null  int64  
 12  PM2.5              120000 non-null  int64  
 13  PM10               0 non-null       float64
dtypes: float64(4), int64(3), object(7)
memory usage: 12.8+ MB
<class 'pandas.core.frame.DataFrame'>
Rang

(None,
                  date    city      station component  value          unit  \
 0  2022-01-01 0:00:00  Dammam   Grantville       NO2     44   mg/mÂ3ŹHigh   
 1  2022-01-01 1:00:00  Dammam  North Erica      PM10    180   mg/mÂ3ŹHigh   
 2  2022-01-01 2:00:00  Riyadh  Lake Mariah       NO2    135   Â5g/mÂ3ŹLow   
 3  2022-01-01 3:00:00  Riyadh   Collinview      PM10     35  Â5g/mÂ3ŹHigh   
 4  2022-01-01 4:00:00  Riyadh  Lake Mariah        CO    290   mg/mÂ3ŹHigh   
 
   indicator periodicity  quarter  temperature (Â0C)  humidity (%)  \
 0     Daily          Q1    36.91              48.23          7.74   
 1     Daily          Q3    32.28              51.47         34.99   
 2     Daily          Q1    27.01              38.85         35.26   
 3     Daily          Q3    33.29              47.80          3.05   
 4     Daily          Q3    15.43              20.38         32.33   
 
    wind_speed (km/h)  PM2.5  PM10  
 0                 17    177   NaN  
 1                 32    19

In [80]:
# Step 1: Clean column names
df_air.rename(columns={"temperature (Â0C)": "temperature", "humidity (%)": "humidity"}, inplace=True)

# Step 2: Drop empty PM10 column
df_air.drop(columns=["PM10"], inplace=True)

# Step 3: Convert date columns to datetime format
df_air["date"] = pd.to_datetime(df_air["date"], errors="coerce")
df_hospital["admission_date"] = pd.to_datetime(df_hospital["admission_date"], errors="coerce")

# Step 4: Extract useful time features
df_air["year"] = df_air["date"].dt.year
df_air["month"] = df_air["date"].dt.month
df_air["day"] = df_air["date"].dt.day
df_air["hour"] = df_air["date"].dt.hour

df_hospital["year"] = df_hospital["admission_date"].dt.year
df_hospital["month"] = df_hospital["admission_date"].dt.month
df_hospital["day"] = df_hospital["admission_date"].dt.day
df_hospital["hour"] = df_hospital["admission_date"].dt.hour

# Step 5: Clean unit column (remove strange characters)
df_air["unit"] = df_air["unit"].str.replace(r"[^\w\s]", "", regex=True).str.strip()

In [81]:
# Check unique values in city columns to compare naming conventions
hospital_cities = df_hospital["hospital_name"].unique()
air_quality_cities = df_air["city"].unique()

# Check date range in both datasets
hospital_date_range = (df_hospital["admission_date"].min(), df_hospital["admission_date"].max())
air_quality_date_range = (df_air["date"].min(), df_air["date"].max())

hospital_cities, air_quality_cities, hospital_date_range, air_quality_date_range

(array(['Mecca City Hospital', 'Dammam General Hospital',
        'Medina Specialist Hospital', 'Dammam Central Hospital',
        'King Saud Hospital', 'Jeddah National Hospital',
        'Riyadh National Hospital', 'Riyadh General Hospital'],
       dtype=object),
 array(['Dammam', 'Riyadh', 'Medina', 'Jeddah', 'Mecca'], dtype=object),
 (Timestamp('2020-01-01 00:00:00'), Timestamp('2024-09-26 23:00:00')),
 (Timestamp('2022-01-01 00:00:00'), Timestamp('2024-09-26 23:00:00')))

In [82]:
# Step 1: Create a mapping of hospital names to their corresponding cities
hospital_to_city = {
    "Mecca City Hospital": "Mecca",
    "Dammam General Hospital": "Dammam",
    "Medina Specialist Hospital": "Medina",
    "Dammam Central Hospital": "Dammam",
    "King Saud Hospital": "Riyadh",
    "Jeddah National Hospital": "Jeddah",
    "Riyadh National Hospital": "Riyadh",
    "Riyadh General Hospital": "Riyadh",
}

# Step 2: Create a new column in the hospital dataset with the mapped city names
df_hospital["city"] = df_hospital["hospital_name"].map(hospital_to_city)

# Step 3: Filter hospital dataset to match air quality date range (2022+)
df_hospital_filtered = df_hospital[df_hospital["admission_date"] >= "2022-01-01"]

In [83]:
# Rename "admission_date" in hospital dataset to match air quality dataset
df_hospital_filtered = df_hospital_filtered.rename(columns={"admission_date": "date"})

# Retry merging with the corrected column names
merged_df_fixed = df_hospital_filtered.merge(df_air, on=["city", "date"], how="left")

# Check merge success
merged_df_fixed.info(), merged_df_fixed.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44285 entries, 0 to 44284
Data columns (total 34 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   date                       44285 non-null  datetime64[ns]
 1   hospital_name              44285 non-null  object        
 2   admission_count            44285 non-null  int64         
 3   condition_type             44285 non-null  object        
 4   patient_age_group          44285 non-null  object        
 5   patient_gender             44285 non-null  object        
 6   readmission_count          44285 non-null  int64         
 7   severity_level             44285 non-null  object        
 8   length_of_stay_avg         44285 non-null  float64       
 9   seasonal_indicator         44285 non-null  object        
 10  comorbid_conditions_count  44285 non-null  int64         
 11  primary_diagnosis_code     44285 non-null  object        
 12  dail

(None,
                  date            hospital_name  admission_count  \
 0 2022-01-01 00:00:00  Riyadh General Hospital                2   
 1 2022-01-01 00:00:00  Riyadh General Hospital                2   
 2 2022-01-01 01:00:00  Dammam General Hospital                1   
 3 2022-01-01 02:00:00  Riyadh General Hospital                3   
 4 2022-01-01 02:00:00  Riyadh General Hospital                3   
 
   condition_type patient_age_group patient_gender  readmission_count  \
 0         Asthma             18-45           Male                  0   
 1         Asthma             18-45           Male                  0   
 2         Asthma             18-45         Female                  0   
 3         Asthma             18-45         Female                  0   
 4         Asthma             18-45         Female                  0   
 
   severity_level  length_of_stay_avg seasonal_indicator  ...  periodicity  \
 0         Severe            3.775284             Winter  ...    

In [85]:
# Step 1: Re-extract date features
merged_df_fixed["month"] = merged_df_fixed["date"].dt.month
merged_df_fixed["day"] = merged_df_fixed["date"].dt.day
merged_df_fixed["hour"] = merged_df_fixed["date"].dt.hour

# Step 2: Create season feature again
merged_df_fixed["season"] = merged_df_fixed["month"].map({
    12: "Winter", 1: "Winter", 2: "Winter",
    3: "Spring", 4: "Spring", 5: "Spring",
    6: "Summer", 7: "Summer", 8: "Summer",
    9: "Fall", 10: "Fall", 11: "Fall"
})

# Step 3: Recreate weekend feature
merged_df_fixed["weekend"] = merged_df_fixed["date"].dt.weekday.isin([4, 5]).astype(int)

# Step 4: Environmental interaction features
merged_df_fixed["temp_humidity_index"] = merged_df_fixed["temperature"] * merged_df_fixed["humidity"]
merged_df_fixed["wind_PM"] = merged_df_fixed["wind_speed"] * merged_df_fixed["PM2.5"]

# Step 5: Encode severity levels
severity_mapping = {"Mild": 1, "Moderate": 2, "Severe": 3}
merged_df_fixed["severity_numeric"] = merged_df_fixed["severity_level"].map(severity_mapping)

# Step 6: Emergency visit ratio
merged_df_fixed["emergency_ratio"] = merged_df_fixed["emergency_visit_count"] / (merged_df_fixed["admission_count"] + 1)

# Check new features
merged_df_fixed[["season", "weekend", "temp_humidity_index", "wind_PM", "severity_numeric", "emergency_ratio"]].head()

KeyError: 'wind_speed'

In [86]:
# Check which columns from the air quality dataset are missing after merging
missing_cols = [col for col in df_air.columns if col not in merged_df_fixed.columns]
missing_cols

['year']

In [87]:
# Fill missing values in air quality columns using the median per city & month
air_quality_cols = ["PM2.5", "temperature", "humidity", "wind_speed", "NO2", "SO2"]

for col in air_quality_cols:
    merged_df_fixed[col] = merged_df_fixed.groupby(["city", "month"])[col].transform(lambda x: x.fillna(x.median()))

# Retry creating wind_PM feature
merged_df_fixed["wind_PM"] = merged_df_fixed["wind_speed"] * merged_df_fixed["PM2.5"]

# Check if missing values are resolved
merged_df_fixed[air_quality_cols].isna().sum()


KeyError: 'Column not found: wind_speed'

In [88]:
# Check all column names in merged dataset
merged_df_fixed.columns

Index(['date', 'hospital_name', 'admission_count', 'condition_type',
       'patient_age_group', 'patient_gender', 'readmission_count',
       'severity_level', 'length_of_stay_avg', 'seasonal_indicator',
       'comorbid_conditions_count', 'primary_diagnosis_code',
       'daily_medication_dosage', 'emergency_visit_count', 'year_x', 'month_x',
       'day_x', 'hour_x', 'city', 'station', 'component', 'value', 'unit',
       'indicator', 'periodicity', 'quarter', 'temperature', 'humidity',
       'wind_speed (km/h)', 'PM2.5', 'year_y', 'month_y', 'day_y', 'hour_y',
       'month', 'day', 'hour', 'season', 'weekend', 'temp_humidity_index'],
      dtype='object')

In [89]:
# Rename wind_speed column for consistency
merged_df_fixed.rename(columns={"wind_speed (km/h)": "wind_speed"}, inplace=True)

# Retry creating wind_PM feature
merged_df_fixed["wind_PM"] = merged_df_fixed["wind_speed"] * merged_df_fixed["PM2.5"]

# Check if missing values are resolved
merged_df_fixed[["wind_speed", "PM2.5", "wind_PM"]].isna().sum()


wind_speed    8619
PM2.5            0
wind_PM       8619
dtype: int64

In [90]:
# Fill missing wind_speed values with median per city & month
merged_df_fixed["wind_speed"] = merged_df_fixed.groupby(["city", "month"])["wind_speed"].transform(lambda x: x.fillna(x.median()))

# Recalculate wind_PM feature
merged_df_fixed["wind_PM"] = merged_df_fixed["wind_speed"] * merged_df_fixed["PM2.5"]

# Check again for missing values
merged_df_fixed[["wind_speed", "PM2.5", "wind_PM"]].isna().sum()


wind_speed    0
PM2.5         0
wind_PM       0
dtype: int64

In [91]:
# Check available columns in the dataset
merged_df_fixed.columns

Index(['date', 'hospital_name', 'admission_count', 'condition_type',
       'patient_age_group', 'patient_gender', 'readmission_count',
       'severity_level', 'length_of_stay_avg', 'seasonal_indicator',
       'comorbid_conditions_count', 'primary_diagnosis_code',
       'daily_medication_dosage', 'emergency_visit_count', 'year_x', 'month_x',
       'day_x', 'hour_x', 'city', 'station', 'component', 'value', 'unit',
       'indicator', 'periodicity', 'quarter', 'temperature', 'humidity',
       'wind_speed', 'PM2.5', 'year_y', 'month_y', 'day_y', 'hour_y', 'month',
       'day', 'hour', 'season', 'weekend', 'temp_humidity_index', 'wind_PM'],
      dtype='object')

In [92]:
# Step 1: Check if NO2 and SO2 exist in the original air quality dataset
missing_pollutants = [col for col in ["NO2", "SO2"] if col not in df_air.columns]

# Step 2: Recreate severity_numeric
severity_mapping = {"Mild": 1, "Moderate": 2, "Severe": 3}
merged_df_fixed["severity_numeric"] = merged_df_fixed["severity_level"].map(severity_mapping)

# Step 3: Recalculate emergency_ratio
merged_df_fixed["emergency_ratio"] = merged_df_fixed["emergency_visit_count"] / (merged_df_fixed["admission_count"] + 1)

# Step 4: Verify if fixes were applied
missing_pollutants, merged_df_fixed[["severity_numeric", "emergency_ratio"]].isna().sum()

(['NO2', 'SO2'],
 severity_numeric    0
 emergency_ratio     0
 dtype: int64)

In [93]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Step 1: Select features and target variable
features = ["temperature", "humidity", "wind_speed", "PM2.5", "NO2", "SO2",
            "season", "weekend", "temp_humidity_index", "wind_PM", "severity_numeric", "emergency_ratio"]

target = "admission_count"  # Predicting hospital admissions

# Step 2: Encode categorical features
merged_df_fixed["season"] = LabelEncoder().fit_transform(merged_df_fixed["season"])

# Step 3: Split data
X = merged_df_fixed[features]
y = merged_df_fixed[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Train models
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
gb_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)

rf_model.fit(X_train, y_train)
gb_model.fit(X_train, y_train)

# Step 5: Make predictions
rf_preds = rf_model.predict(X_test)
gb_preds = gb_model.predict(X_test)

# Step 6: Evaluate models
rf_mae, rf_mse, rf_r2 = mean_absolute_error(y_test, rf_preds), mean_squared_error(y_test, rf_preds), r2_score(y_test, rf_preds)
gb_mae, gb_mse, gb_r2 = mean_absolute_error(y_test, gb_preds), mean_squared_error(y_test, gb_preds), r2_score(y_test, gb_preds)

(rf_mae, rf_mse, rf_r2), (gb_mae, gb_mse, gb_r2)


KeyError: "['NO2', 'SO2'] not in index"

In [94]:
# Check available columns in the dataset
merged_df_fixed.columns

Index(['date', 'hospital_name', 'admission_count', 'condition_type',
       'patient_age_group', 'patient_gender', 'readmission_count',
       'severity_level', 'length_of_stay_avg', 'seasonal_indicator',
       'comorbid_conditions_count', 'primary_diagnosis_code',
       'daily_medication_dosage', 'emergency_visit_count', 'year_x', 'month_x',
       'day_x', 'hour_x', 'city', 'station', 'component', 'value', 'unit',
       'indicator', 'periodicity', 'quarter', 'temperature', 'humidity',
       'wind_speed', 'PM2.5', 'year_y', 'month_y', 'day_y', 'hour_y', 'month',
       'day', 'hour', 'season', 'weekend', 'temp_humidity_index', 'wind_PM',
       'severity_numeric', 'emergency_ratio'],
      dtype='object')

In [95]:
# Step 1: Check if NO2 and SO2 exist in the original air quality dataset
missing_pollutants = [col for col in ["NO2", "SO2"] if col not in df_air.columns]

# Step 2: Recreate severity_numeric
severity_mapping = {"Mild": 1, "Moderate": 2, "Severe": 3}
merged_df_fixed["severity_numeric"] = merged_df_fixed["severity_level"].map(severity_mapping)

# Step 3: Recalculate emergency_ratio
merged_df_fixed["emergency_ratio"] = merged_df_fixed["emergency_visit_count"] / (merged_df_fixed["admission_count"] + 1)

# Step 4: Verify if fixes were applied
missing_pollutants, merged_df_fixed[["severity_numeric", "emergency_ratio"]].isna().sum()

(['NO2', 'SO2'],
 severity_numeric    0
 emergency_ratio     0
 dtype: int64)

In [96]:
# Re-import necessary libraries
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor



# Step 1: Reinitialize models
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
gb_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)

# Step 1: Update feature list (remove NO2 and SO2)
features = ["temperature", "humidity", "wind_speed", "PM2.5",  
            "season", "weekend", "temp_humidity_index", "wind_PM", "severity_numeric", "emergency_ratio"]

# Step 2: Split data again
X = merged_df_fixed[features]
y = merged_df_fixed[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Retrain models
rf_model.fit(X_train, y_train)
gb_model.fit(X_train, y_train)

# Step 4: Make predictions again
rf_preds = rf_model.predict(X_test)
gb_preds = gb_model.predict(X_test)

# Step 5: Evaluate models again
rf_mae, rf_mse, rf_r2 = mean_absolute_error(y_test, rf_preds), mean_squared_error(y_test, rf_preds), r2_score(y_test, rf_preds)
gb_mae, gb_mse, gb_r2 = mean_absolute_error(y_test, gb_preds), mean_squared_error(y_test, gb_preds), r2_score(y_test, gb_preds)

(rf_mae, rf_mse, rf_r2), (gb_mae, gb_mse, gb_r2)

ValueError: Input X contains NaN.
GradientBoostingRegressor does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [97]:
# Step 1: Check which columns have missing values
missing_values = merged_df_fixed[features].isna().sum()
missing_values

temperature               0
humidity                  0
wind_speed                0
PM2.5                     0
season                    0
weekend                   0
temp_humidity_index    8619
wind_PM                   0
severity_numeric          0
emergency_ratio           0
dtype: int64

In [98]:
# Step 1: Fill missing values with median
merged_df_fixed["temp_humidity_index"].fillna(merged_df_fixed["temp_humidity_index"].median(), inplace=True)

# Step 2: Verify if all missing values are handled
merged_df_fixed[features].isna().sum()

/tmp/ipykernel_5664/2197966736.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df_fixed["temp_humidity_index"].fillna(merged_df_fixed["temp_humidity_index"].median(), inplace=True)


temperature            0
humidity               0
wind_speed             0
PM2.5                  0
season                 0
weekend                0
temp_humidity_index    0
wind_PM                0
severity_numeric       0
emergency_ratio        0
dtype: int64

In [99]:
# Step 1: Split data again (now clean)
X = merged_df_fixed[features]
y = merged_df_fixed[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 2: Retrain models
rf_model.fit(X_train, y_train)
gb_model.fit(X_train, y_train)

# Step 3: Make predictions
rf_preds = rf_model.predict(X_test)
gb_preds = gb_model.predict(X_test)

# Step 4: Evaluate models
rf_mae, rf_mse, rf_r2 = mean_absolute_error(y_test, rf_preds), mean_squared_error(y_test, rf_preds), r2_score(y_test, rf_preds)
gb_mae, gb_mse, gb_r2 = mean_absolute_error(y_test, gb_preds), mean_squared_error(y_test, gb_preds), r2_score(y_test, gb_preds)

(rf_mae, rf_mse, rf_r2), (gb_mae, gb_mse, gb_r2)

((0.6697335369437356, 1.079400667682128, 0.4545148700053926),
 (0.6814494974942417, 1.0259197867844583, 0.4815419287632279))

In [100]:
from sklearn.model_selection import GridSearchCV

# Define parameter grids for both models
rf_param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20],
    'min_samples_split': [2, 5]
}

gb_param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.05, 0.1],
    'max_depth': [3, 5],
    'min_samples_split': [2, 5]
}

# Initialize GridSearchCV for both models
rf_grid_search = GridSearchCV(RandomForestRegressor(random_state=42), rf_param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
gb_grid_search = GridSearchCV(GradientBoostingRegressor(random_state=42), gb_param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)

# Run GridSearchCV for Random Forest
rf_grid_search.fit(X_train, y_train)
best_rf = rf_grid_search.best_estimator_

# Run GridSearchCV for Gradient Boosting
gb_grid_search.fit(X_train, y_train)
best_gb = gb_grid_search.best_estimator_

# Get best parameters
rf_best_params = rf_grid_search.best_params_
gb_best_params = gb_grid_search.best_params_

rf_best_params, gb_best_params

({'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 200},
 {'learning_rate': 0.05,
  'max_depth': 5,
  'min_samples_split': 2,
  'n_estimators': 100})

In [101]:
# Import necessary libraries
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Train Random Forest with best parameters
rf_best = RandomForestRegressor(max_depth=10, min_samples_split=5, n_estimators=200, random_state=42)
rf_best.fit(X_train, y_train)

# Train Gradient Boosting with best parameters
gb_best = GradientBoostingRegressor(learning_rate=0.05, max_depth=5, min_samples_split=2, n_estimators=100, random_state=42)
gb_best.fit(X_train, y_train)

GradientBoostingRegressor(learning_rate=0.05, max_depth=5, random_state=42)

In [102]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Make predictions
rf_preds = rf_best.predict(X_test)
gb_preds = gb_best.predict(X_test)

# Evaluate Random Forest
rf_mae = mean_absolute_error(y_test, rf_preds)
rf_mse = mean_squared_error(y_test, rf_preds)
rf_r2 = r2_score(y_test, rf_preds)

# Evaluate Gradient Boosting
gb_mae = mean_absolute_error(y_test, gb_preds)
gb_mse = mean_squared_error(y_test, gb_preds)
gb_r2 = r2_score(y_test, gb_preds)

# Print results
print("Random Forest Performance:")
print(f"MAE: {rf_mae}, MSE: {rf_mse}, R²: {rf_r2}\n")

print("Gradient Boosting Performance:")
print(f"MAE: {gb_mae}, MSE: {gb_mse}, R²: {gb_r2}")

Random Forest Performance:
MAE: 0.6348383893409685, MSE: 1.0024937634551936, R²: 0.4933804867367567

Gradient Boosting Performance:
MAE: 0.650770566900301, MSE: 1.0047270918433497, R²: 0.49225185354001244
